<a href="https://colab.research.google.com/github/sumankanukollu/denseDepthMaps_S15/blob/master/EVA4_S15A_Suman_800K_customDataset_prepScript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

```Data written to Google Drive is first written to the disk attached to the VM, so the maximum size is limited by that.
The google.colab.drive module has a recently-added flush_and_unmount() function that you can use to sync data written to the local VM's disk cache of your Drive-mounted folder back to Google Drive, after which a "Reset all runtimes" (from the Runtime menu) will get you a fresh VM.
We're working on making all this more transparent.```

In [0]:
import torch
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

PyTorch 1.5.0+cu101 CPU


In [0]:
import sys,os
from pathlib import Path

homepath = Path('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms')
sys.path.append(str(homepath))

os.chdir(homepath)
!pwd
!ls

/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms
bg  fg	mask


# Import required modules

In [0]:
import sys,os,pdb,shutil,time
from pathlib import Path
from zipfile import ZipFile 

import numpy as np
from PIL import Image
import random
import copy
import PIL

from datetime import datetime

import gc
gc.enable()
gc.get_threshold()

(700, 10, 10)

# Method to create overlay Images 

In [0]:
def startBatch(s,e,outputZip= 'output_0.zip'):
    gc.enable()
    gc.garbage
    start_time = datetime.now()
    black = np.zeros((224,224))
    try:
        with ZipFile(outputZip,'a') as z:
            for i in range(s,e): 
                start = 1 
                print('Using background : {}'.format(i))
                bgi =f'{homepath}/bg/{str(i)}_bg.jpg'
                bg 	= Image.open(f'{homepath}/bg/{str(i)}_bg.jpg')
                for j in  range(1,101): 
                    fg 		= Image.open(f'{homepath}/fg/{str(j)}_fg.jpg')
                    fgi 	= f'{homepath}/fg/{str(j)}_fg.jpg'
                    m 		= Image.open(f'{homepath}/mask/{str(j)}_fg_mask.jpg')
                    mi 		= f'{homepath}/mask/{str(j)}_fg_mask.jpg'
                    for k in range(1,21): 
                        r1 = random.randint(1, bg.size[0]-fg.size[0])
                        r2 = random.randint(1, bg.size[0]-fg.size[0])

                        bg_1_1, fg_t, m_t = copy.deepcopy(bg),copy.deepcopy(fg),copy.deepcopy(m)
                        bg_1_2, fg_f, m_f = copy.deepcopy(bg),copy.deepcopy(fg),copy.deepcopy(m)

                        fg_f = fg_f.transpose(Image.FLIP_LEFT_RIGHT) #flip fg image
                        m_f	 = m_f.transpose(Image.FLIP_LEFT_RIGHT) #flip mask

                        black_img1 	= Image.fromarray(black,mode='L')
                        black_img1.paste(m_t,(r1,r2), m_t.split()[0])
                        fname = os.path.basename(bgi).split('.')[0]+'_'+str(start)+'_fg.jpg'
                        #black_img1.save(os.path.join(homepath,'bg_fg_mask',fname), optimize=True, quality=60)
                        black_img1.save('tmp_bg_fg_mask.jpg', optimize=True, quality=60)
                        z.write('tmp_bg_fg_mask.jpg',os.path.join('bg_fg_mask_1',fname))

                        black_img2 = Image.fromarray(black,mode='L')
                        black_img2.paste(m_f,(r1,r2), m_f.split()[0])
                        fname = os.path.basename(bgi).split('.')[0]+'_'+str(start)+'_fg.jpg'
                        #black_img2.save(os.path.join(homepath,'bg_fgFlip_mask',fname), optimize=True, quality=60)
                        black_img2.save('tmp_bg_fgFlip_mask.jpg', optimize=True, quality=60)
                        z.write('tmp_bg_fgFlip_mask.jpg',os.path.join('bg_fgFlip_mask_1',fname))


                        bg_1_1.paste(fg_t,(r1,r2),m_t.split()[0])
                        fname = os.path.basename(bgi).split('.')[0]+'_'+str(start)+'_fg.jpg'
                        bg_1_1.save('tmp_bg_fg.jpg', optimize=True, quality=60)
                        z.write('tmp_bg_fg.jpg',os.path.join('bg_fg_1',fname))

                        bg_1_2.paste(fg_f,(r1,r2),m_f.split()[0])
                        fname = os.path.basename(bgi).split('.')[0]+'_'+str(start)+'_fgFlip.jpg'
                        #bg_1_2.save(os.path.join(homepath,'bg_fgFlip',fname), optimize=True, quality=60)
                        bg_1_2.save('tmp_bg_fgFlip.jpg', optimize=True, quality=60)
                        z.write('tmp_bg_fgFlip.jpg',os.path.join('bg_fgFlip_1',fname))

                        [print(start) if start/500 in range(1000) else None]
                        start+=1
    except Exception as e:
        print(str(e))
        print('Fg img : {} and BG Image : {}'.format(j,i))
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))



# 1-10 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(1,11,outputZip= 'output_0.zip')

Using background : 1
500
1000
1500
2000
Using background : 2
500
1000
1500
2000
Using background : 3
500
1000
1500
2000
Using background : 4
500
1000
1500
2000
Using background : 5
500
1000
1500
2000
Using background : 6
500
1000
1500
2000
Using background : 7
500
1000
1500
2000
Using background : 8
500
1000
1500
2000
Using background : 9
500
1000
1500
2000
Using background : 10
500
1000
1500
2000
Duration: 0:10:43.054554


## 1-10 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_0.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_0
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# 11-20 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(11,21,outputZip= 'output_1.zip')

Using background : 11
500
1000
1500
2000
Using background : 12
500
1000
1500
2000
Using background : 13
500
1000
1500
2000
Using background : 14
500
1000
1500
2000
Using background : 15
500
1000
1500
2000
Using background : 16
500
1000
1500
2000
Using background : 17
500
1000
1500
2000
Using background : 18
500
1000
1500
2000
Using background : 19
500
1000
1500
2000
Using background : 20
500
1000
1500
2000
Duration: 0:10:23.393650


## 11-20 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_1.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_1
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# 21-30 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(21,31,outputZip= 'output_2.zip')

Using background : 21
500
1000
1500
2000
Using background : 22
500
1000
1500
2000
Using background : 23
500
1000
1500
2000
Using background : 24
500
1000
1500
2000
Using background : 25
500
1000
1500
2000
Using background : 26
500
1000
1500
2000
Using background : 27
500
1000
1500
2000
Using background : 28
500
1000
1500
2000
Using background : 29
500
1000
1500
2000
Using background : 30
500
1000
1500
2000
Duration: 0:10:12.103429


## 21-30 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
print('Started at : {}'.format(start_time))
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_2.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_2
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# 31-40 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(31,41,outputZip= 'output_3.zip')

Using background : 31
500
1000
1500
2000
Using background : 32
500
1000
1500
2000
Using background : 33
500
1000
1500
2000
Using background : 34
500
1000
1500
2000
Using background : 35
500
1000
1500
2000
Using background : 36
500
1000
1500
2000
Using background : 37
500
1000
1500
2000
Using background : 38
500
1000
1500
2000
Using background : 39
500
1000
1500
2000
Using background : 40
500
1000
1500
2000
Duration: 0:10:01.763599


## 31-40 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
print('Started at : {}'.format(start_time))
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_3.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_3
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# 41-50 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(41,51,outputZip= 'output_4.zip')

Using background : 41
500
1000
1500
2000
Using background : 42
500
1000
1500
2000
Using background : 43
500
1000
1500
2000
Using background : 44
500
1000
1500
2000
Using background : 45
500
1000
1500
2000
Using background : 46
500
1000
1500
2000
Using background : 47
500
1000
1500
2000
Using background : 48
500
1000
1500
2000
Using background : 49
500
1000
1500
2000
Using background : 50
500
1000
1500
2000
Duration: 0:10:05.061687


## 41-50 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
print('Started at : {}'.format(start_time))
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_4.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_4
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# 51-60 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(51,61,outputZip= 'output_5.zip')

Using background : 51
500
1000
1500
2000
Using background : 52
500
1000
1500
2000
Using background : 53
500
1000
1500
2000
Using background : 54
500
1000
1500
2000
Using background : 55
500
1000
1500
2000
Using background : 56
500
1000
1500
2000
Using background : 57
500
1000
1500
2000
Using background : 58
500
1000
1500
2000
Using background : 59
500
1000
1500
2000
Using background : 60
500
1000
1500
2000
Duration: 0:10:08.953407


## 51-60 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
print('Started at : {}'.format(start_time))
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_5.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_5
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# 61-70 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(61,71,outputZip= 'output_6.zip')

Using background : 61
500
1000
1500
2000
Using background : 62
500
1000
1500
2000
Using background : 63
500
1000
1500
2000
Using background : 64
500
1000
1500
2000
Using background : 65
500
1000
1500
2000
Using background : 66
500
1000
1500
2000
Using background : 67
500
1000
1500
2000
Using background : 68
500
1000
1500
2000
Using background : 69
500
1000
1500
2000
Using background : 70
500
1000
1500
2000
Duration: 0:10:17.870592


## 61-70 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
print('Started at : {}'.format(start_time))
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_6.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_6
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# 71-80 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(71,81,outputZip= 'output_7.zip')

Using background : 71
500
1000
1500
2000
Using background : 72
500
1000
1500
2000
Using background : 73
500
1000
1500
2000
Using background : 74
500
1000
1500
2000
Using background : 75
500
1000
1500
2000
Using background : 76
500
1000
1500
2000
Using background : 77
500
1000
1500
2000
Using background : 78
500
1000
1500
2000
Using background : 79
500
1000
1500
2000
Using background : 80
500
1000
1500
2000
Duration: 0:10:18.061205


## 71-80 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
print('Started at : {}'.format(start_time))
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_7.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_7
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# 81-90 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(81,91,outputZip= 'output_8.zip')

Using background : 81
500
1000
1500
2000
Using background : 82
500
1000
1500
2000
Using background : 83
500
1000
1500
2000
Using background : 84
500
1000
1500
2000
Using background : 85
500
1000
1500
2000
Using background : 86
500
1000
1500
2000
Using background : 87
500
1000
1500
2000
Using background : 88
500
1000
1500
2000
Using background : 89
500
1000
1500
2000
Using background : 90
500
1000
1500
2000
Duration: 0:10:10.873715


## 81-90 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
print('Started at : {}'.format(start_time))
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_8.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_8
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# 91-100 BG Images Overlay Foreground Images on Background

In [0]:
startBatch(91,101,outputZip= 'output_9.zip')

Using background : 91
500
1000
1500
2000
Using background : 92
500
1000
1500
2000
Using background : 93
500
1000
1500
2000
Using background : 94
500
1000
1500
2000
Using background : 95
500
1000
1500
2000
Using background : 96
500
1000
1500
2000
Using background : 97
500
1000
1500
2000
Using background : 98
500
1000
1500
2000
Using background : 99
500
1000
1500
2000
Using background : 100
500
1000
1500
2000
Duration: 0:09:55.325788


## 91-100 BG Images unzip output folders

In [0]:
gc.enable()
gc.garbage
start_time = datetime.now()
print('Started at : {}'.format(start_time))
!unzip -q /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_9.zip -d /content/drive/My\ Drive/EVA4/S15A_RCNN/original_jpg_imgs/output_9
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

# Final Number of Images count in GDrive

In [0]:
import os
os.chdir('/content/drive/My Drive/EVA4/S15A_RCNN_RGB/bg_fg_mask_jpgIms')

In [0]:
import zipfile
from PIL import Image

count=0
for d in range(0,10):
    imgzip = zipfile.ZipFile("output_%i.zip"%d)
    inflist = imgzip.infolist()
    count += len(inflist)
    print('Number of images in folder {} is  {}'.format("output_%i.zip"%d,len(inflist)))

print('Total number of Images are : {}'.format(count))

Number of images in folder output_0.zip is  80000
Number of images in folder output_1.zip is  80000
Number of images in folder output_2.zip is  80000
Number of images in folder output_3.zip is  80000
Number of images in folder output_4.zip is  80000
Number of images in folder output_5.zip is  80000
Number of images in folder output_6.zip is  80000
Number of images in folder output_7.zip is  80000
Number of images in folder output_8.zip is  80000
Number of images in folder output_9.zip is  80000
Total number of Images are : 800000


# Read Images directly from zip folder

In [0]:
start =1
for f in inflist[:1]:
    ifile = imgzip.open(f)
    img = Image.open(ifile)
    print(img)
    #img.save('test_%i'%start+'.jpg')
    img.show()
    start +=1

In [0]:
import os
from pathlib import Path

homepath = Path('/content/drive/My Drive/EVA4/S15A_RCNN/original_jpg_imgs/')
os.chdir(homepath)
!ls output_0

In [0]:
count=0
for p in "output_0  output_1  output_2  output_3  output_4 ".split():
    for d in "bg_fg_1  bg_fgFlip_1  bg_fgFlip_mask_1	bg_fg_mask_1".split():
        count +=len(os.listdir(os.path.join(p,d)))
    print('File in path :{} is {}'.format(p,len(os.listdir(os.path.join(p,d)))))
print(count)